In [ ]:
%load_ext autoreload
%autoreload 2

# Imports

### Standard imports

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
import pickle
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Important for multiprocessing.
import torch
torch.set_num_threads(1)

### Plots

In [ ]:
from plot import get_3d_subplot_axs
from plot import get_figsize, set_figsize

default_w, default_h = get_figsize()

### Lattice imports

In [ ]:
from collections import OrderedDict
from gridsearch import experiment, load_experiment

# Create NARMA dataset

In [ ]:
import dataset as ds

u_train, y_train = ds.NARMA(sample_len = 2000)
u_test, y_test = ds.NARMA(sample_len = 3000)
dataset = [u_train, y_train, u_test, y_test]
ds.dataset = dataset

# Waxman graph generation

In [ ]:
from matrix import waxman
from plot import scatter_3d

In [ ]:
from matrix import euclidean

# We need names for the distance functions to select them from a pandas
# dataframe later.
euc = euclidean
def inv(x, y): return 1/euclidean(x, y)
def inv_squared(x, y): return 1/euclidean(x, y)**2

### Waxman visualization

In [ ]:
for z_frac in [0.0, 0.5, 1.0]:
    G = waxman(n=200, alpha=1.0, beta=1.0, z_frac=z_frac)
    scatter_3d(G)

### Waxman performance with increasing fraction of 3d nodes

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['dist_function'] = [inv]
params['hidden_nodes'] = np.arange(20, 90, 10)
params['z_frac'] = np.arange(0.0, 1.2, 0.2)
params['directed'] = [True, False]
waxman_nrmse_df = experiment(esn_nrmse, params, runs=20)
waxman_nrmse_df.to_pickle('experiments/waxman_nrmse.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_nrmse_df' not in locals():
    waxman_nrmse_df = load_experiment('experiments/waxman_nrmse.pkl')

df = waxman_nrmse_df.loc[waxman_nrmse_df['directed'] == False]
groupby = ['hidden_nodes', 'z_frac']

axes    = ['hidden_nodes', 'z_frac', 'esn_nrmse']
agg     = ['mean', 'min']
zlim    = [(0.40, 0.80), (0.40, 0.80)]
azim    = [160, 160]
title   = 'Increasing fraction of 3d nodes'

plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title=title)

A hypothesis as to what we're seeing from the *min* plot is that there is little  
noticeable difference with increasing *z_frac*. Additionally, we do not see the  
expected performance increase when adding more hidden nodes to the reservoir.  

An initial for why more nodes does not matter is that there is a cap on what the  
reservoir may be capable of learning with only positive weights, especially when  
the network is fully connected. In fact, more nodes makes it more likely for the  
network to be "unstable", explaining the higher performance variance seen in the  
leftmost plot. Since all weights are positive: does the scaling for spectral  
radius purposes have to be tighter?  

### Distribution of reservoir weights with different distance functions

The first thing we want to address is how the distribution of weights differ  
with different distance functions for the generated Waxman graph.  

In [ ]:
from ESN import ESN
from plot import plot_esn_weight_hist

bins = 40
params = {
    'hidden_nodes': 150,
}

plt.title('Reference ESN')
plot_esn_weight_hist(params, n_bins=bins, show=True)

set_figsize(14, 6)

params = {
    'directed': False,
    'w_res_type': 'waxman',
}

hidden_nodes = [50, 150, 150, 150]
dist_functions = [euc, euc, inv, inv_squared]
titles = ['50 nodes euc', '150 nodes euc', '150 nodes inv', '150 nodes inv^2']

for z_frac in [0.0, 1.0]:
    fig, axs = plt.subplots(1, 4)

    params['z_frac'] = z_frac
    plt.suptitle('z_frac:' + str(z_frac))

    for i in range(len(axs)):
        params['hidden_nodes'] = hidden_nodes[i]
        params['dist_function'] = dist_functions[i]

        ax = axs[i]
        ax.set_title(titles[i])
        plot_esn_weight_hist(params, n_bins=bins, ax=ax, show=False)

    plt.show()

set_figsize(default_w, default_h)

We see the clear difference between the functions *d*, *1/d* and *1/d^*; we can  
almost see the functions in the distributions themselves.  

When the weight function moves from d to 1/d to 1/d, we see that we are shifting  
the probability distribution towards many lower weights, with just a few big  
ones, which is as expected.  

### Waxman performance with changing weight/distance function

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['z_frac'] = np.arange(0.0, 1.2, 0.2)
params['hidden_nodes'] = np.arange(20, 90, 10)
params['dist_function'] = [euc, inv, inv_squared]
waxman_dist_df = experiment(esn_nrmse, params, runs=20)
waxman_dist_df.to_pickle('experiments/waxman_dist.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_dist_df' not in locals():
    waxman_dist_df = load_experiment('experiments/waxman_dist.pkl')

waxman_dist_df['dist_function'] = waxman_dist_df['dist_function'].apply(
    lambda f: f.__name__ if not isinstance(f, str) else f
)

euc_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == euc.__name__]
inv_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == inv.__name__]
inv_squared_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == inv_squared.__name__]

groupby = ['hidden_nodes', 'z_frac']
axes    = ['hidden_nodes', 'z_frac', 'esn_nrmse']
agg     = ['mean', 'min']
titles  = ['euc', 'inv', 'inv^2']
labels  = ['euc', 'inv', 'inv^2']
zlim    = (0.3, 1.0)
azim    = 110

for i, df in enumerate([euc_df, inv_df, inv_squared_df]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title=titles[i])

ax = get_3d_subplot_axs(1)[0]
show = [False, False, True]
for i, df in enumerate([euc_df, inv_df, inv_squared_df]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, azim=azim, zlim=zlim,
                    title='min for all dist functions', agg=['min'], ax=ax, label=labels[i],
                    show=show[i])

It does seem like the different distance functions may all achieve about the  
same performance in terms of error rate on NARMA10, giving a slight edge to the  
inverse functions. However, the latter ones also introduce the instability we  
see disappear when we use a minimum aggregation instead. The weights in general  
are _smaller_, as can be seen in the weight distribution plots. First thoughts:  
can this be remedied with input scalings? It is clear that _z\_frac_ has little  
impact on performance, so let's change it to input scaling.  

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['input_scaling'] = np.arange(0.1, 2.1, 0.1)
params['hidden_nodes'] = np.arange(20, 90, 10)
params['dist_function'] = [euc, inv, inv_squared]
waxman_is_df = experiment(esn_nrmse, params, runs=20)
waxman_is_df.to_pickle('experiments/waxman_is.pkl')

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['input_scaling'] = np.arange(0.1, 2.1, 0.1)
params['hidden_nodes'] = [80]
params['dist_function'] = [euc, inv, inv_squared]
waxman_is_mc_df = experiment(esn_mc, params, runs=20)
waxman_is_mc_df.to_pickle('experiments/waxman_is_mc.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_is_df' not in locals():
    waxman_is_df = load_experiment('experiments/waxman_is.pkl')

waxman_is_df['dist_function'] = waxman_is_df['dist_function'].apply(
    lambda f: f.__name__ if not isinstance(f, str) else f
)

euc_df = waxman_is_df.loc[waxman_is_df['dist_function'] == euc.__name__]
inv_df = waxman_is_df.loc[waxman_is_df['dist_function'] == inv.__name__]
inv_squared_df = waxman_is_df.loc[waxman_is_df['dist_function'] == inv_squared.__name__]

groupby = ['hidden_nodes', 'input_scaling']
axes    = ['hidden_nodes', 'input_scaling', 'esn_nrmse']
agg     = ['min']
titles  = ['euc', 'inv', 'inv^2']
zlim    = (0.5, 0.7)
azim    = 20

for i, df in enumerate([euc_df, inv_df, inv_squared_df]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title=titles[i])

In [ ]:
from plot import plot_df_trisurf

if 'waxman_is_mc_df' not in locals():
    waxman_is_mc_df = load_experiment('experiments/waxman_is_mc.pkl')

waxman_is_mc_df['dist_function'] = waxman_is_mc_df['dist_function'].apply(
    lambda f: f.__name__ if not isinstance(f, str) else f
)

euc_df = waxman_is_mc_df.loc[waxman_is_mc_df['dist_function'] == euc.__name__]
inv_df = waxman_is_mc_df.loc[waxman_is_mc_df['dist_function'] == inv.__name__]
inv_squared_df = waxman_is_mc_df.loc[waxman_is_mc_df['dist_function'] == inv_squared.__name__]

labels = ['euc', 'inv', 'inv^2']
plt.title('STM (MC) - 80 hidden nodes')
plt.xlabel('Input scaling')
plt.ylabel('STM (MC)')

for i, df in enumerate([euc_df, inv_df, inv_squared_df]):
    grouped_df = df.groupby(['input_scaling']).mean().reset_index()
    plt.plot(grouped_df['input_scaling'], grouped_df['esn_mc'], label=labels[i])

plt.legend()
plt.show()

Input scaling seems to have a quite big impact on the different distance  
functions. In fact, 1/d^2 has a minimum NRMSE with correct input scaling of 0.5,  
which is quite a lot lower than the original distance function d.  

In the STM plot, we see that 1d^2 consistently has better memory performance  
than the other distance functions, even before changing the input scaling. That  
much. An analysis of the lyapunov spectrum could prove interesting.  

Can we see any patterns if we look the output weights of the network for tests  
of increasing memory capacity?  

In [ ]:
from metric import memory_capacity
from plot import scatter_3d

params = {
    'w_res_type': 'waxman',
    'hidden_nodes': 80,
    'dist_function': inv_squared,
    'input_scaling': 0.1,
}

esn = ESN(**params)
mc = memory_capacity(esn)
print('Memory capacity:', mc)

for i in range(3, 9):
    weights = esn.w_outs[i].abs().data.numpy()
    weights /= weights.max()
    title = 'Hidden weights for STM length: ' + str(i)
    scatter_3d(esn.G, title=title, cols=weights)

I see no concernable pattern, which is perhaps as expected from a fully  
connected network.  

The question remains: what is the difference that makes the inverted functions  
increasingly performant on NARMA/memory? Are the internal node activations  
relevant?  

In [ ]:
from metric import evaluate_esn

# For comparison to a default network.
params = { 'hidden_nodes': 80 }
esn = ESN(**params)
nrmse = evaluate_esn(ds.dataset, esn)
plt.title('Reference NRMSE: ' + str(nrmse))
plt.plot(esn.X)
plt.show()

# With different distance functions.
dist_functions = [euc, inv, inv_squared]
params = {
    'w_res_type': 'waxman',
    'hidden_nodes': 80,
    'input_scaling': 0.1,
}

for f in dist_functions:
    params['dist_function'] = f
    esn = ESN(**params)
    nrmse = evaluate_esn(ds.dataset, esn)

    plt.title(f.__name__ + ' NRMSE: ' + str(nrmse))
    plt.plot(esn.X)
    plt.show()

Note: this creates a whole new network every time instead of re-using the  
existing network but changing the weighting scheme, as to try to understand the  
whole picture.  

From running this a few times, I notice that the times when the network simply  
does not work at all, the root cause is that a big majority of the network nodes  
are either *all positive* or *all negative*, there is no balance.  

There is a tendency here: the networks that seem to perform well also seem to  
resemble the default first ESN network in terms of "look", but not so in the  
magnitude of the activations -- which is lowered by the spectral radius. This  
seems to indicate that the initial spectral radius of the waxman networks are  
quite a lot higher than that of default initializations.  

### Moving waxman performance towards Echo State Networks

Recall the results we got for the original Waxman graph for the 1/d distance  
function:  

In [ ]:
if 'waxman_dist_df' not in locals():
    waxman_dist_df = load_experiment('experiments/waxman_dist.pkl')

waxman_dist_df['dist_function'] = waxman_dist_df['dist_function'].apply(
    lambda f: f.__name__ if not isinstance(f, str) else f
)

inv_df = waxman_dist_df.loc[waxman_dist_df['dist_function'] == inv.__name__]

groupby = ['hidden_nodes', 'z_frac']
axes    = ['hidden_nodes', 'z_frac', 'esn_nrmse']
agg     = ['mean', 'min']
labels  = ['euc', 'inv', 'inv^2']
zlim    = (0.3, 1.0)
azim    = 110

plot_df_trisurf(df=inv_df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title='inv')

We will attempt to make the performance of the Waxman graph equivalent to that  
of the Echo State Network by:  

* Introducing negative weights
* Making the graph directed

#### Increasing fraction of negative weights

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['dist_function'] = [inv]
params['z_frac'] = [1.0]
params['hidden_nodes'] = np.arange(20, 90, 10)
params['sign_frac'] = np.arange(0.0, 0.55, 0.05)
params['directed'] = [True, False]
waxman_sign_df = experiment(esn_nrmse, params, runs=20)
waxman_sign_df.to_pickle('experiments/waxman_sign.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'waxman_sign_df' not in locals():
    waxman_sign_df = load_experiment('experiments/waxman_sign.pkl')

undirected_df = waxman_sign_df.loc[waxman_sign_df['directed'] == False]
directed_df = waxman_sign_df.loc[waxman_sign_df['directed'] == True]

groupby = ['hidden_nodes', 'sign_frac']
axes    = ['hidden_nodes', 'sign_frac', 'esn_nrmse']
agg     = ['mean', 'min']
title   = 'Increasing fraction of negative weights'
zlim    = (0.3, 1.0)
azim    = 70

plot_df_trisurf(df=undirected_df, groupby=groupby, axes=axes,
                agg=agg, azim=azim, zlim=zlim, title=title)

#### Directed Waxman graphs

In [ ]:
title = 'Effect of directedness'

plot_df_trisurf(df=directed_df, groupby=groupby, axes=axes,
                agg=agg, azim=azim, zlim=zlim, title=title)

#### Comparison to Echo State Networks

In [ ]:
%%script false --no-raise-error

params = OrderedDict()
params['hidden_nodes'] = np.arange(20, 90, 10)
waxman_esn_df = experiment(esn_nrmse, params, runs=20)
waxman_esn_df.to_pickle('experiments/waxman_default_esn.pkl')

In [ ]:
if 'waxman_esn_df' not in locals():
    waxman_esn_df = load_experiment('experiments/waxman_default_esn.pkl')

waxman_esn_df_max = waxman_esn_df.copy()
waxman_esn_df_max['sign_frac'] = 0.5
waxman_esn_df_min = waxman_esn_df.copy()
waxman_esn_df_min['sign_frac'] = 0.0
waxman_esn_df = pd.concat([waxman_esn_df_max, waxman_esn_df_min])

labels  = ['+Negative weights', '+Directed', 'Echo State Network']
show    = [False, False, True]
azim    = 45
elev    = 20

ax = get_3d_subplot_axs(1)[0]
ax.invert_yaxis()
for i, df in enumerate([undirected_df, directed_df, waxman_esn_df]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, azim=azim, elev=elev,
                    zlim=zlim, title='min', agg=['min'], ax=ax, label=labels[i],
                    show=show[i])

Adding the possibility of having negative weights makes for a big difference,  
especially when also combined with a directed network. In fact, we end up with  
performance that very closely resembles that of ESNs at the end:  

In [ ]:
undirected = undirected_df.loc[undirected_df['sign_frac'] == 0.5]
directed = directed_df.loc[directed_df['sign_frac'] == 0.5]
esn = waxman_esn_df.loc[waxman_esn_df['sign_frac'] == 0.5]

labels = ['Undirected', 'Directed', 'Echo State Network']

for i, df in enumerate([undirected, directed, esn]):
    df = df.groupby(['hidden_nodes', 'sign_frac']).min().reset_index()
    plt.plot(df['hidden_nodes'], df['esn_nrmse'], label=labels[i])

plt.legend()
plt.show()

Dale et al. investigates this a bit in their paper: «It then becomes clear  
that how weights are structured and directed, controlling information flow,  
has a greater affect on quality of the network. This supports similar results  
using hierarchical networks, where structure and number of parameters also  
significantly impact performance [6].»  

Lastly, for the similarly performing networks in the previous graph, their  
weight distributions look like (exemplified with the undirected graph, as their  
distributions will be the same, except a directed graph would be symmetric):  

In [ ]:
from plot import plot_vector_hist, plot_esn_weight_hist
from ESN import ESN

bins = 200
hidden_nodes = 400

params = OrderedDict()
params['hidden_nodes'] = hidden_nodes
plt.title('Echo State Network reservoir weight distribution')
plot_esn_weight_hist(params, n_bins=bins, m=20.0, show=True)

params = OrderedDict()
params['w_res_type'] = 'waxman'
params['dist_function'] = inv
params['z_frac'] = 1.0
params['hidden_nodes'] = hidden_nodes
params['sign_frac'] = 0.5
params['directed'] = False
esn = ESN(**params)
M = esn.w_res.data.numpy()

# Masking to remove diagonal (i.e. 0-elements) of reservoir weights.
M = M[~np.eye(M.shape[0], dtype=bool)].reshape(M.shape[0], -1)

weights = M.flatten()
plt.title('Waxman graph reservoir weight distribution (network)')
plot_vector_hist(weights, n_bins=bins, m=4.0, show=True)

weights = M[0].flatten()
plt.title('Waxman graph reservoir weight distribution (single node)')
plot_vector_hist(weights, n_bins=bins, m=4.0, show=True)

First thoughts: does this perhaps fit some known distribution (perhaps something  
similar to a power law)?  

Now: does this distribution also scale as well with increasing hidden nodes as  
the Echo State Network?  

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

hidden_nodes = np.arange(20, 260, 10)

params = OrderedDict()
params['w_res_type'] = ['waxman']
params['dist_function'] = [inv]
params['z_frac'] = [1.0]
params['hidden_nodes'] = hidden_nodes
params['sign_frac'] = [0.5]
params['directed'] = [True]
waxman_dist_hn_df = experiment(esn_nrmse, params, runs=20)
waxman_dist_hn_df.to_pickle('experiments/waxman_dist_hn.pkl')

params = OrderedDict()
params['hidden_nodes'] = hidden_nodes
esn_hn_df = experiment(esn_nrmse, params, runs=20)
esn_hn_df.to_pickle('experiments/esn_hn.pkl')

In [ ]:
if 'waxman_dist_hn_df' not in locals():
    waxman_dist_hn_df = load_experiment('experiments/waxman_dist_hn.pkl')

if 'esn_hn_df' not in locals():
    esn_hn_df = load_experiment('experiments/esn_hn.pkl')

gdf = waxman_dist_hn_df.groupby(['hidden_nodes', 'w_res_type']).mean().reset_index()
plt.plot(gdf['hidden_nodes'], gdf['esn_nrmse'], label='Modified Waxman graph')

gdf = esn_hn_df.groupby(['hidden_nodes']).mean().reset_index()
plt.plot(gdf['hidden_nodes'], gdf['esn_nrmse'], label='Echo State Network')

plt.xlabel('Hidden nodes')
plt.ylabel('NRMSE')

plt.legend()
plt.show()

So it seems that this symmetric weight distribution performs reasonably  
equivalent to the standard ESN. Does it provide an additional robustness? 

It should not for dead nodes, as with a global connectivity, this is equivalent  
to simply removing the node (i.e. decrementing the amount of hidden nodes).  

What about sporadically firing nodes, or nodes that constantly produce noise?  

I found little interesting in an introductory study of the difference in  
robustness attributed to the difference in weight distributions. I might look  
deeper into it later.  

# Lattice/tiling experiments (sq, rect, hex, tri)

### Plots of lattices

In [ ]:
from ESN import ESN
from plot import plot_lattice

set_figsize(14, 6)

esn_square = ESN(hidden_nodes=25, w_res_type='tetragonal')
esn_hex = ESN(hidden_nodes=25, w_res_type='hexagonal')
esn_tri = ESN(hidden_nodes=25, w_res_type='triangular')
esn_rect = ESN(hidden_nodes=25, w_res_type='rectangular', rect_ratio=2.0)

G_square = esn_square.G
G_hex = esn_hex.G
G_tri = esn_tri.G
G_rect = esn_rect.G

fig, axs = plt.subplots(2, 2)
ax1, ax2, ax3, ax4 = axs[0, 0], axs[0, 1], axs[1, 0], axs[1, 1]
plot_lattice(G_square, title='Square', ax=ax1, show=False)
plot_lattice(G_hex, title='Hexagonal', ax=ax2, show=False)
plot_lattice(G_tri, title='Triangular', ax=ax3, show=False)
plot_lattice(G_rect, title='Rectangular', ax=ax4, show=True)

set_figsize(default_w, default_h)

### Growing neighborhoods

In [ ]:
from ESN import ESN
from plot import plot_lattice
from matrix import grow_neighborhoods

set_figsize(14, 6)

esn_sq = ESN(hidden_nodes=25, w_res_type='tetragonal')
G_sq = esn_sq.G
titles = ['"Exponential" growth', 'Default growth']

for n_plt in [0, 1]:
    _G_sq = G_sq.copy()

    fig, axs = plt.subplots(2, 2)
    ax1, ax2, ax3, ax4 = axs[0, 0], axs[0, 1], axs[1, 0], axs[1, 1]
    axs = [ax1, ax2, ax3, ax4]

    for i, ax in enumerate(axs):
        plot_lattice(_G_sq, title='Square', ax=ax, neigh_color=True, show=False)

        # The second time around we don't "exponentially" grow the neighborhood,
        # but instead only use the original adjacency matrix to grow the
        # neighborhood `l` times.
        if n_plt == 0:
            grow_neighborhoods(_G_sq, dist_function=inv)
        else:
            _G_sq = G_sq.copy()
            grow_neighborhoods(_G_sq, dist_function=inv, l=i+1)

    plt.suptitle(titles[n_plt])

set_figsize(default_w, default_h)

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [9, 16, 25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
params['grow_neigh'] = list(range(1, 9))
params['dist_function'] = [inv]
lattice_grow_df = experiment(esn_nrmse, params)
lattice_grow_df.to_pickle('experiments/lattice_grow_neigh.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'lattice_grow_df' not in locals():
    lattice_grow_df = load_experiment('experiments/lattice_grow_neigh.pkl')

groupby = ['hidden_nodes', 'grow_neigh']
axes    = ['hidden_nodes', 'grow_neigh', 'esn_nrmse']
agg     = ['mean', 'min']
titles  = ['Square', 'Hexagonal', 'Triangular']
zlim    = (0.5, 0.8)
azim    = -60
elev    = 20

for i, w_res_type in enumerate(['tetragonal', 'hexagonal', 'triangular']):
    df = lattice_grow_df.loc[lattice_grow_df['w_res_type'] == w_res_type]
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, azim=azim, elev=elev,
                    zlim=zlim, title=titles[i])

There is a definite performance penalty with increasing sizes of node  
neighborhood.  

In [ ]:
set_figsize(14, 6)

params = {
    'w_res_type': 'tetragonal',
    'hidden_nodes': 14*14,
    'dist_function': inv,
    'grow_neigh': 0,
    'readout': 'pinv',
}

esn = ESN(**params)
nrmse = evaluate_esn(ds.dataset, esn, plot=True)
print('NRMSE', nrmse )

set_figsize(default_w, default_h)

### Lattice NRMSE

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [9, 16, 25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
nrmse_df = experiment(esn_nrmse, params)
nrmse_df.to_pickle('experiments/lattice_nrmse.pkl')

In [ ]:
if 'nrmse_df' not in locals():
    nrmse_df = load_experiment('experiments/lattice_nrmse.pkl')

grouped_df = nrmse_df.groupby(['hidden_nodes', 'w_res_type']).mean().reset_index()

tetragonal = grouped_df.loc[grouped_df['w_res_type'] == 'tetragonal']
hexagonal = grouped_df.loc[grouped_df['w_res_type'] == 'hexagonal']
triangular = grouped_df.loc[grouped_df['w_res_type'] == 'triangular']

plt.plot(tetragonal['hidden_nodes'], tetragonal['esn_nrmse'], label='sq')
plt.plot(hexagonal['hidden_nodes'], hexagonal['esn_nrmse'], label='hex')
plt.plot(triangular['hidden_nodes'], triangular['esn_nrmse'], label='tri')

plt.legend(fancybox=False, loc='upper right', bbox_to_anchor=(1.0, 1.0))
plt.ylabel('NRMSE')
plt.xlabel('Hidden nodes')

plt.show()

### Lattice STM

#### Measuring the STM (MC) of lattices

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [9, 16, 25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
mc_df = experiment(esn_mc, params)
mc_df.to_pickle('experiments/lattice_mc.pkl')

In [ ]:
if 'mc_df' not in locals():
    mc_df = load_experiment('experiments/lattice_mc.pkl')

grouped_df = mc_df.groupby(['hidden_nodes', 'w_res_type']).mean().reset_index()

tetragonal = grouped_df.loc[grouped_df['w_res_type'] == 'tetragonal']
hexagonal = grouped_df.loc[grouped_df['w_res_type'] == 'hexagonal']
triangular = grouped_df.loc[grouped_df['w_res_type'] == 'triangular']

plt.plot(tetragonal['hidden_nodes'], tetragonal['esn_mc'], label='sq')
plt.plot(hexagonal['hidden_nodes'], hexagonal['esn_mc'], label='hex')
plt.plot(triangular['hidden_nodes'], triangular['esn_mc'], label='tri')

plt.legend(fancybox=False, loc='upper right', bbox_to_anchor=(1.0, 1.0))
plt.ylabel('MC')
plt.xlabel('Hidden nodes')

plt.show()

#### Is there some pattern in the STM in lattices?

First: how does the recall of the network look?

In [ ]:
from metric import memory_capacity
from plot import plot_lattice

params = {
    'w_res_type': 'hexagonal',
    'hidden_nodes': 49,
    'input_scaling': 0.1,
    'periodic': False,
}

esn = ESN(**params)
mc = memory_capacity(esn)
print('Memory capacity:', mc)

plt.title(f'Hexagonal lattice memory capacity for {esn.hidden_nodes} hidden nodes')
plt.xlabel('Signal delay')
plt.ylabel('Recall ability')
plt.plot(esn.mcs, marker='*')
plt.show()

There was no pattern for the positions of high weighted hidden nodes for the  
Waxman graphs, what about highly regular structures/networks?  

In [ ]:
for i in range(3, int(mc)):
    weights = esn.w_outs[i].abs().data.numpy()
    weights /= weights.max()
    title = 'Hidden weights for STM length: ' + str(i)
    plot_lattice(esn.G, title=title, cols=weights)

*An important note here: this works quite a bit worse if the input distribution  
is fixed; the uniform distribution is crucial here.*  

Quite interesting! A lot of the memory capacity seems to be due to the edges of  
the lattice, seemingly because it is aperiodic. Making the graph periodic tanks  
the memory capacity quite a little bit, which is investigated a little bit  
further below.  

Additionally, big weights in the output layer are often surrounded by other big  
weights, which is an interesting indication of the flow of information in the  
network.  

Lastly, there is not necessarsily anything indicating that clusters of weights  
used to recall n is close to weights used to recall n-1, which is slightly  
surprising.  

What about the activations of the seemingly most important hidden nodes?

In [ ]:
set_figsize(12, 5)

length = 200
n_max = 3
n_min = 3

for i in range(3, 6):
    weights = esn.w_outs[i].abs().data.numpy()
    min_elems = np.argpartition(weights, n_min)[:n_min]
    max_elems = np.argpartition(weights, -n_max)[-n_max:]

    min_activations = esn.X_train[:length, min_elems]
    max_activations = esn.X_train[:length, max_elems]

    fig, (ax1, ax2) = plt.subplots(1, 2)
    plt.suptitle('Activations for nodes of biggest w_out weights for STM length: ' + str(i))
    for ax in [ax1, ax2]:
        ax.set_ylim(-0.08, 0.08)

    ax1.plot(min_activations)
    ax2.plot(max_activations)

set_figsize(default_w, default_h)

### Lattice input scaling

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['input_scaling'] = np.insert(np.arange(0.05, 2.05, 0.05), 0, 0.01, axis=0)
params['w_res_type'] = [None, 'tetragonal', 'hexagonal', 'triangular']
waxman_is_df = experiment(esn_mc, params)
is_df.to_pickle('experiments/lattice_input_scaling.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'is_df' not in locals():
    is_df = load_experiment('experiments/lattice_input_scaling.pkl')

reference_esn = is_df.loc[is_df['w_res_type'].isnull()]
tetragonal = is_df.loc[is_df['w_res_type'] == 'tetragonal']
hexagonal = is_df.loc[is_df['w_res_type'] == 'hexagonal']
triangular = is_df.loc[is_df['w_res_type'] == 'triangular']

groupby = ['hidden_nodes', 'input_scaling']
axes    = ['input_scaling', 'hidden_nodes', 'esn_mc']
agg     = ['mean', 'max']
titles  = ['Reference ESN', 'Square', 'Hexagonal', 'Triangular']
zlims   = [(5, 50)] + [(5, 20)]*3
azim    = 70

for i, df in enumerate([reference_esn, tetragonal, hexagonal, triangular]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, agg=agg, zlim=zlims[i], title=titles[i])

### Using a rectangular lattice instead of square

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['w_res_type'] = ['rectangular']
params['rect_ratio'] = np.arange(0.1, 3.1, 0.1)
rect_df = experiment(esn_nrmse, params)
rect_df.to_pickle('experiments/lattice_rect.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'rect_df' not in locals():
    rect_df = load_experiment('experiments/lattice_rect.pkl')

groupby = ['hidden_nodes', 'rect_ratio']
axes    = ['hidden_nodes', 'rect_ratio', 'esn_nrmse']
agg     = ['mean', 'min']
titles  = title = 'Effect of rectangular lattice'
zlim    = (0.5, 0.62)
azim    = 40

plot_df_trisurf(df=rect_df, groupby=groupby, axes=axes, agg=agg, azim=azim, zlim=zlim, title=title)

### Periodic lattice

In [ ]:
%%script false --no-raise-error

from metric import esn_mc

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['periodic'] = [True, False]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
periodic_df = experiment(esn_mc, params)
periodic_df.to_pickle('experiments/periodic_lattice.pkl')

In [ ]:
if 'periodic_df' not in locals():
    periodic_df = load_experiment('experiments/periodic_lattice.pkl')

gdf = periodic_df.groupby(['hidden_nodes', 'periodic', 'w_res_type']).mean().reset_index()

from collections import defaultdict
dfs = defaultdict(lambda: [])

plt.title('Periodic lattices')
plt.ylabel('MC')
plt.xlabel('Hidden nodes')
colors = ['green', 'red', 'blue']

for c, l in enumerate(['tetragonal' , 'hexagonal', 'triangular']):
    dfs[l].append(gdf.loc[(gdf['w_res_type'] == l) & (gdf['periodic'] == False)])
    dfs[l].append(gdf.loc[(gdf['w_res_type'] == l) & (gdf['periodic'] == True)])

    for i, p in enumerate(['aperiodic', 'periodic']):
        x = dfs[l][i]['hidden_nodes']
        y = dfs[l][i]['esn_mc']
        label = l + ' ' + p
        linestyle = '-' if i == 0 else '--'
        plt.plot(x, y, label=label, linestyle=linestyle, color=colors[c])

plt.legend(fancybox=False, loc='lower right', bbox_to_anchor=(1.0, 0.0))
plt.show()

### Lattice with a fraction of negative weights

In [ ]:
esn = ESN(hidden_nodes=25, w_res_type='hexagonal', sign_frac=0.5)
plot_lattice(esn.G, edge_color=True, directed=True)

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['hidden_nodes'] = [25, 36, 49, 64, 81]
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
params['sign_frac'] = np.arange(0.0, 0.55, 0.05)
lattice_sign_df = experiment(esn_nrmse, params)
lattice_sign_df.to_pickle('experiments/lattice_sign.pkl')

In [ ]:
if 'lattice_sign_df' not in locals():
    lattice_sign_df = load_experiment('experiments/lattice_sign.pkl')

tetragonal = lattice_sign_df.loc[lattice_sign_df['w_res_type'] == 'tetragonal']
hexagonal = lattice_sign_df.loc[lattice_sign_df['w_res_type'] == 'hexagonal']
triangular = lattice_sign_df.loc[lattice_sign_df['w_res_type'] == 'triangular']

groupby = ['hidden_nodes', 'sign_frac']
axes    = ['hidden_nodes', 'sign_frac', 'esn_nrmse']
agg     = ['mean', 'min']
zlim    = (0.55, 0.7)
azim    = -50
titles  = ['tetragonal', 'hexagonal', 'triangular']

for i, df in enumerate([tetragonal, hexagonal, triangular]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, azim=azim, agg=agg, zlim=zlim, title=titles[i])

No difference is obtained by setting some edges to have negative weights.  

### Lattice with a fraction of directed edges

In [ ]:
esn = ESN(hidden_nodes=25, w_res_type='hexagonal', dir_frac=0.5)
plot_lattice(esn.G)

In [ ]:
# %%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
params['hidden_nodes'] = [25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225]
params['dir_frac'] = np.arange(0.0, 1.1, 0.1)
lattice_dir_df = experiment(esn_nrmse, params, runs=50)
lattice_dir_df.to_pickle('experiments/lattice_dir.pkl')

In [ ]:
from plot import plot_df_trisurf

if 'lattice_dir_df' not in locals():
    lattice_dir_df = load_experiment('experiments/lattice_dir.pkl')

tetragonal = lattice_dir_df.loc[lattice_dir_df['w_res_type'] == 'tetragonal']
hexagonal = lattice_dir_df.loc[lattice_dir_df['w_res_type'] == 'hexagonal']
triangular = lattice_dir_df.loc[lattice_dir_df['w_res_type'] == 'triangular']

groupby = ['hidden_nodes', 'dir_frac']
axes    = ['hidden_nodes', 'dir_frac', 'esn_nrmse']
agg     = ['mean', 'min']
zlim    = (0.25, 0.7)
azim    = 45
titles  = ['tetragonal', 'hexagonal', 'triangular']

for i, df in enumerate([tetragonal, hexagonal, triangular]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, azim=azim, agg=agg, zlim=zlim, title=titles[i])

We are below the strictly linear regime of NARMA10! That is interesting.

### Lattice negative weights+directed

In [ ]:
%%script false --no-raise-error

from metric import esn_nrmse

params = OrderedDict()
params['w_res_type'] = ['tetragonal', 'hexagonal', 'triangular']
params['hidden_nodes'] = [100]
params['dir_frac'] = np.arange(0.0, 1.1, 0.1)
params['sign_frac'] = np.arange(0.0, 0.55, 0.05)
lattice_neg_dir_df = experiment(esn_nrmse, params, runs=20)
lattice_neg_dir_df.to_pickle('experiments/lattice_neg_dir.pkl')

In [ ]:
if 'lattice_neg_dir_df' not in locals():
    lattice_neg_dir_df = load_experiment('experiments/lattice_neg_dir.pkl')

tetragonal = lattice_neg_dir_df.loc[lattice_neg_dir_df['w_res_type'] == 'tetragonal']
hexagonal = lattice_neg_dir_df.loc[lattice_neg_dir_df['w_res_type'] == 'hexagonal']
triangular = lattice_neg_dir_df.loc[lattice_neg_dir_df['w_res_type'] == 'triangular']

groupby = ['sign_frac', 'dir_frac']
axes    = ['sign_frac', 'dir_frac', 'esn_nrmse']
agg     = ['mean', 'min']
zlim    = (0.5, 0.65)
azim    = 45
titles  = ['tetragonal', 'hexagonal', 'triangular']

for i, df in enumerate([tetragonal, hexagonal, triangular]):
    plot_df_trisurf(df=df, groupby=groupby, axes=axes, azim=azim, agg=agg, zlim=zlim, title=titles[i])

### Spectral radius with directed lattice

In [ ]:
%%script false --no-raise-error

from metric import evaluate_esn
from ESN import ESN

params = OrderedDict()
params['w_res_type'] = 'tetragonal'
params['hidden_nodes'] = 144

params['dir_frac'] = 0.0
while True:
    undir_esn = ESN(**params)
    undir_nrmse = evaluate_esn(ds.dataset, undir_esn)
    if undir_nrmse < 0.55:
        break

params['dir_frac'] = 1.0
while True:
    dir_esn = ESN(**params)
    dir_nrmse = evaluate_esn(ds.dataset, dir_esn)
    if dir_nrmse < 0.35:
        break

pickle.dump(dir_esn, open('models/dir_esn.pkl', 'wb'))
pickle.dump(undir_esn, open('models/undir_esn.pkl', 'wb'))

In [ ]:
dir_esn = pickle.load(open('models/dir_esn.pkl', 'rb'))
undir_esn = pickle.load(open('models/undir_esn.pkl', 'rb'))

print('NRMSE:')
print(' undirected:', undir_nrmse)
print(' directed:  ', dir_nrmse)

print()
print('Original spectral radius')
print(' undirected:', undir_esn.org_spectral_radius)
print(' directed:  ', dir_esn.org_spectral_radius)

Huh! The directed square lattice with 144 hidden nodes is as good as the Echo  
State Networks of the same size. This is surprising and promising. Perhaps it is  
necessary to design physical reservoirs in simulation first, if one is to  
consider the flow of information as such?  

Do the activations look wildly different between directed and undirected graphs?  
For example, is the directed graph perhaps able to reach the nonlinear part of  
tanh?  

In [ ]:
l = 100
fig, (ax1, ax2) = plt.subplots(1, 2)
plt.suptitle('Activations for an undirected vs. directed lattice')

ax1.plot(undir_esn.X[undir_esn.washout:undir_esn.washout+l])
ax1.set_title('Undirected')

ax2.plot(dir_esn.X[dir_esn.washout:dir_esn.washout+l])
ax2.set_title('Directed')

plt.show()

It is difficult to differentiate the two: this seems to be a common occurrence.  

Is there perhaps a correlation between the spectral radius of the original  
adjacency matrix (before scaling), and the NRMSE performance of the lattice?  

In [ ]:
%%script false --no-raise-error

from metric import evaluate_esn
from ESN import ESN

params = OrderedDict()
params['w_res_type'] = 'tetragonal'
params['hidden_nodes'] = 144
params['dir_frac'] = 1.0

nrmses = []
srs = []
for i in range(100):
    esn = ESN(**params)
    nrmse = evaluate_esn(ds.dataset, esn)
    nrmses.append(nrmse)
    srs.append(esn.org_spectral_radius)

pickle.dump(nrmses, open('models/dir_nrmses.pkl', 'wb'))
pickle.dump(srs, open('models/dir_srs.pkl', 'wb'))

In [ ]:
nrmses = pickle.load(open('models/dir_nrmses.pkl', 'rb'))
srs = pickle.load(open('models/dir_srs.pkl', 'rb'))

plt.xlabel('Original spectral radius')
plt.ylabel('Evaluated NRMSE')

plt.scatter(srs, nrmses)
plt.show()

I see little/no correlation between the original spectral radius and the  
evaluated NRMSE, so this is unlikely to be the culprit.  

What about how the well-performing lattice _looks_?  

In [ ]:
from plot import plot_lattice
plot_lattice(dir_esn.G)

With directedness: what is the distributions of in-degree/out-degree with  
directed lattices? Can we color the lattices in some way to give insights?  

An idea: what about average path length?  